## TS2 - Problem wieloagentowy - Paweł Malisz, Mikołaj Białek - Tic-Tac-Toe

### 1. Opis gry

Kółko i krzyżyk to jedna z najstarszych dwuosobowych gier ręcznych na świecie. Gracze starają się ułożyć przypisany sobie znak (X lub O) w jednym rzędzie, kolumnie bądź skosie. Wygrywa ten, który połączy w ten sposób trzy znaki (w tradycyjnej odmianie).

### 2. Cel ćwiczenia

Celem naszego ćwiczenia jest rozwiązanie problemu wieloagentowego z biblioteki PettingZoo używając uczenia mszynowego w języku Python. 

Przedstawiony zostanie algorytm DQN (Deep Q-learning) na platformie Tianshou wraz z krzywą uczenia.

### 3. Opis problemu

Na grę składają się dwa ciągłe zbiory akcji i obserwacji. Zbiór akcji zawiera pojedyńczą liczbę oznaczającą ruch gracza - jest to liczba od 0-8, który symbolizuję numer pola. 

0 | 3 | 6

1 | 4 | 7

2 | 5 | 8

Natomiast zbiór obserwacji składa się z siatki 3x3 oraz dwóch wymiarów, przy czym pierwszy oznacza rozmienieczenie X, a drugi rozmieszczenie O. Obydwa wymiary zawierają zbiór 0 i 1. W pierwszym 1 jest w momencie, gdy występuje na niej X, natomst 0, gdy tego X tam nie ma. Analogicznie w drugim wymiarze dla O.

### 4. Rozwiązanie problemu

Zacznijmy od funkcji definiującej nasze środowisko i po krótcę ją opiszmy. Nasze klasyczne środowisko zawiera dwóch agentów.

In [ ]:
from tianshou.env.pettingzoo_env import PettingZooEnv
from pettingzoo.classic import tictactoe_v3

def _get_env():
    return PettingZooEnv(tictactoe_v3.env())

Stwórzmy nasze środowisko, 100 próbek uczących i 100 testowych, musimy przekształcić na obiekt `DummyVectorEnv`.

In [ ]:
from tianshou.env import DummyVectorEnv

train_envs = DummyVectorEnv([_get_env for _ in range(100)])
test_envs = DummyVectorEnv([_get_env for _ in range(100)])

Teraz stwórzmy funkcję dla naszych agentów, zrobimy tak, by `player_1` był naszym uczeniem maszynowym, a `player_0` podejmował losowe ruchy.

Opis kodu:
- `player_0` = `agent_opponent` - struktura klasy `RandomPolicy()`,
- `player_1` = `agent_learn` - struktura klasy `DQNPolicy()`, która przyjmuje takie wartości jak:
    - `model` - struktura w tym przypadku klasy `Net()`, opisuje nam stosowaną sieć neuronową, domyślnie z funkcją aktywacji ReLu, która zawiera kształt obserwacji na wejściu, kształt akcji na wyjściu, 4 warstwy ukryte po 128 neuronów oraz device, który wykorzystuje rdzenie cuda karty graficznej, jeśli te są dostępne,
    - `optim` - czyli optymalizator, domnyslnie sotuje się Adam, więc taki zostawiamy,
    - `discount_factor` - stosowany w AI, im większy tym teoretycznie lepiej,
    - `estimation_step` - przewidywana ilość ruchów (w naszym przypadku 3 - potrzebne by szybko wygrać),
    - `target_update_freq` - określa częstotliwość aktualizacji tzw. target network (sieci docelowej) w stosunku do policy network (sieci polityki).

Później łączymy dwie polityki w jedną `MultiAgentPolicyManager()`.

In [ ]:
import torch

import gymnasium as gym
from typing import Optional, Tuple
from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager, RandomPolicy, PPOPolicy
from tianshou.utils.net.common import Net

def _get_agents(
    agent_learn: Optional[BasePolicy] = None,
    agent_opponent: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None,
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    env = _get_env()
    observation_space = (
        env.observation_space["observation"]
        if isinstance(env.observation_space, gym.spaces.Dict)
        else env.observation_space
    )
    state_shape = observation_space.shape or observation_space.n
    action_shape = env.action_space.shape or env.action_space.n
    if agent_learn is None:
        net = Net(
            state_shape = state_shape,
            action_shape = action_shape,
            hidden_sizes = [128, 128, 128, 128],
            device = "cuda" if torch.cuda.is_available() else "cpu",
        ).to("cuda" if torch.cuda.is_available() else "cpu")
        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=1e-4)
        agent_learn = DQNPolicy(
            model = net,
            optim = optim,
            discount_factor = 0.9,
            estimation_step = 3,
            target_update_freq = 320,
        )

    if agent_opponent is None:
        agent_opponent = RandomPolicy()

    agents = [agent_opponent, agent_learn]
    print(agents)
    policy = MultiAgentPolicyManager(agents, env)
    return policy, optim, env.agents

Następnie inicjalizujemy naszych agentów, oraz ustawiamy Collectory, czyli obiekt `Collector` z biblioteki `Tianshou`. Służy onm do zbierania danych treningowych oraz testowych w procesie uczenia ze wzmocnieniem. Służy do interakcji z środowiskiem, gromadzenia trajektorii agentów oraz przechowywania tych trajektorii w pamięci podręcznej w celu późniejszego wykorzystania w procesie uczenia.

In [ ]:
from tianshou.data import Collector, VectorReplayBuffer

policy, optim, agents = _get_agents()
policy.train()

train_collector = Collector(policy, train_envs, VectorReplayBuffer(20000, len(train_envs)))
test_collector = Collector(policy, test_envs)

train_collector.reset()
train_collector.collect(n_step=len(train_envs)*64)

Teraz etap uczenia, służy do niego funkcja `offpolicy_trainer()` z biblioteki `tianshou`.

In [ ]:
from tianshou.trainer import offpolicy_trainer
import tianshou as ts

from torch.utils.tensorboard import SummaryWriter
import os

logger = ts.utils.TensorboardLogger(SummaryWriter('log/dqn'))

def save_best_fn(policy):
    model_save_path = os.path.join("log", "ttt", "dqn", "policy.pth")
    os.makedirs(os.path.join("log", "ttt", "dqn"), exist_ok=True)
    torch.save(policy.policies[agents[1]].state_dict(), model_save_path)

def stop_fn(mean_rewards):
    return mean_rewards >= 0.6 # end, when the win rate is equal or greater than 0.6

def train_fn(epoch, env_step):
    policy.policies[agents[1]].set_eps(0.1)

def test_fn(epoch, env_step):
    policy.policies[agents[1]].set_eps(0.05)

def reward_metric(rews):
    return rews[:, 1]

result = offpolicy_trainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=50,
    step_per_epoch=1000,
    step_per_collect=10,
    episode_per_test=100,
    batch_size=64,
    train_fn=train_fn,
    test_fn=test_fn,
    stop_fn=stop_fn,
    save_best_fn=save_best_fn,
    update_per_step=0.1,
    test_in_train=False,
    reward_metric=reward_metric,
    logger=logger
)
print(f'Finished training! Use {result["duration"]}')

Aby zobaczyć krzywe uczenia, musimy odpalić nasze logi za pomocą poniższej komendy w terminalu:

`tensorboard --logdir log`

![alt text](images/train.png "Wykres na zbiorze uczącym.")

![alt text](images/test.png "Wykres na zbiorze testowym.")

Wytestowanie naszego uczenia na jednej przykładowej grze.

In [ ]:
policy.eval()
my_env = PettingZooEnv(tictactoe_v3.env(render_mode="human"))
my_env = DummyVectorEnv([lambda: my_env])
collector = Collector(policy, my_env, exploration_noise=True)
collector.collect(n_episode=1, render=0.1)